In [5]:
import pandas as pd
import numpy as np
import cv2
from decimal import Decimal, getcontext
getcontext().prec = 2


In [6]:
data = pd.read_csv('ground truth.csv')
preds = pd.read_csv('preds.csv')
data.head()


,image_name,class,xmin,ymin,xmax,ymax
0,img1,person,123,233,183,367
1,img2,motorcycle,276,202,454,310
2,img3,motorcycle,378,204,456,266
3,img4,motorcycle,309,101,429,343
4,img5,motorcycle,71,252,163,348


In [7]:
data['xmin_pred'] = preds.xmin
data['ymin_pred'] = preds.ymin
data['xmax_pred'] = preds.xmax
data['ymax_pred'] = preds.ymax

In [8]:

data.head()

,image_name,class,xmin,ymin,xmax,ymax,xmin_pred,ymin_pred,xmax_pred,ymax_pred
0,img1,person,123,233,183,367,130,253,169,370
1,img2,motorcycle,276,202,454,310,341,209,466,295
2,img3,motorcycle,378,204,456,266,372,211,460,268
3,img4,motorcycle,309,101,429,343,325,157,409,361
4,img5,motorcycle,71,252,163,348,26,275,160,371


In [9]:
def IOU(df):
    # determining the minimum and maximum -coordinates of the intersection rectangle
    xmin_inter = max(df.xmin, df.xmin_pred)
    ymin_inter = max(df.ymin, df.ymin_pred)
    xmax_inter = min(df.xmax, df.xmax_pred)
    ymax_inter = min(df.ymax, df.ymax_pred)
 
    # calculate area of intersection rectangle
    inter_area = max(0, xmax_inter - xmin_inter + 1) * max(0, ymax_inter - ymin_inter + 1)
 
    # calculate area of actual and predicted boxes
    actual_area = (df.xmax - df.xmin + 1) * (df.ymax - df.ymin + 1)
    pred_area = (df.xmax_pred - df.xmin_pred + 1) * (df.ymax_pred - df.ymin_pred+ 1)
 
    # computing intersection over union
    iou = inter_area / float(actual_area + pred_area - inter_area)
 
    # return the intersection over union value
    return iou

In [10]:
# creating a evaluation table
eval_table = pd.DataFrame()
eval_table['image_name'] = data.image_name

#applying IOU function over each image in the dataframe
eval_table['IOU'] = data.apply(IOU, axis = 1)

# creating column 'TP/FP' which will store TP for True positive and FP for False positive
# if IOU is greater than 0.5 then TP else FP
eval_table['TP/FP'] = eval_table['IOU'].apply(lambda x: 'TP' if x>=0.5 else 'FP')

In [11]:
# calculating Precision and recall

Precision = []
Recall = []

TP = FP = 0
FN = len(eval_table['TP/FP']== 'TP')
for index , row in eval_table.iterrows():     
    
    if row.IOU > 0.5:
        TP =TP+1
    else:
        FP =FP+1    
    

    try:
        
        AP = TP/(TP+FP)
        Rec = TP/(TP+FN)
    except ZeroDivisionError:
        
        AP = Recall = 0.0
    
    Precision.append(AP)
    Recall.append(Rec)

In [12]:
eval_table['Precision'] = Precision
eval_table['Recall'] = Recall
eval_table

,image_name,IOU,TP/FP,Precision,Recall
0,img1,0.550569,TP,1.000000,0.090909
1,img2,0.482510,FP,0.500000,0.090909
2,img3,0.774103,TP,0.666667,0.166667
3,img4,0.513853,TP,0.750000,0.230769
4,img5,0.430901,FP,0.600000,0.230769
5,img6,0.577086,TP,0.666667,0.285714
6,img7,0.592355,TP,0.714286,0.333333
7,img8,0.503012,TP,0.750000,0.375000
8,img9,0.283827,FP,0.666667,0.375000
9,img10,0.625393,TP,0.700000,0.411765


In [13]:
#calculating Interpolated Precision
eval_table['IP'] = eval_table.groupby('Recall')['Precision'].transform('max')

In [14]:
eval_table

,image_name,IOU,TP/FP,Precision,Recall,IP
0,img1,0.550569,TP,1.000000,0.090909,1.000000
1,img2,0.482510,FP,0.500000,0.090909,1.000000
2,img3,0.774103,TP,0.666667,0.166667,0.666667
3,img4,0.513853,TP,0.750000,0.230769,0.750000
4,img5,0.430901,FP,0.600000,0.230769,0.750000
5,img6,0.577086,TP,0.666667,0.285714,0.666667
6,img7,0.592355,TP,0.714286,0.333333,0.714286
7,img8,0.503012,TP,0.750000,0.375000,0.750000
8,img9,0.283827,FP,0.666667,0.375000,0.750000
9,img10,0.625393,TP,0.700000,0.411765,0.700000


In [15]:
prec_at_rec = []

for recall_level in np.linspace(0.0, 1.0, 11):
    try:
        x = eval_table[eval_table['Recall'] >= recall_level]['Precision']
        prec = max(x)
    except:
        prec = 0.0
    prec_at_rec.append(prec)
avg_prec = np.mean(prec_at_rec)
print('11 point precision is ', prec_at_rec)
print('\nmap is ', avg_prec)

11 point precision is  [1.0, 0.75, 0.75, 0.75, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

map is  0.3590909090909091
